# 4. Comparación de Modelos de Detección

Este notebook evalúa los 6 modelos entrenados en los datasets de prueba Aéreo y Acuático.

# 1. Importación de librerias y modelos

In [1]:
import os
import pandas as pd
from ultralytics import YOLO
import torch

def clear_gpu_cache():
    if torch.cuda.is_available():
        device = torch.cuda.current_device()
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats(device)
        print(f"Caché de la GPU limpiada en dispositivo {device}.")
    else:
        print("CUDA no está disponible.")
clear_gpu_cache()

CUDA no está disponible.


In [2]:
# --- 1. Configuración de Rutas ---

# Busca automáticamente el directorio de modelos
if os.path.exists('entrenados'):
    MODELS_DIR = 'entrenados'
elif os.path.exists('modelos_entrenados'):
    MODELS_DIR = 'modelos_entrenados'
else:
    MODELS_DIR = '' # No encontrado

ACUATICO_YAML = './dataset_yolo/dataset.yml'

models_to_evaluate = {
    'Aereo-Nano': 'modelo-aereo-n.pt',
    'Aereo-Medium': 'modelo-aereo-m.pt',
    'Aereo-Large': 'modelo-aereo-l.pt',
    'Acuatico-Nano': 'modelo-acuatico-n.pt',
    'Acuatico-Medium': 'modelo-acuatico-m.pt',
    'Acuatico-Large': 'modelo-acuatico-l.pt',
    'Mixto-Nano': 'modelo-mixto-n.pt',
    'Mixto-Medium': 'modelo-mixto-m.pt',
    'Mixto-Large': 'modelo-mixto-l.pt',
}

# Verificaciones
if not MODELS_DIR:
    print("ADVERTENCIA: No se encontró el directorio 'entrenados' ni 'modelos_entrenados'. No se podrá continuar.")
else:
    print(f"Directorio de modelos: '{MODELS_DIR}'")
print('Configuración de rutas completada.')

Directorio de modelos: 'modelos_entrenados'
Configuración de rutas completada.


## 2. Ejecución de Evaluaciones

Se evaluarán todos los modelos en ambos conjuntos de datos. Este proceso puede tardar varios minutos.

In [3]:
def run_evaluation(dataset_yaml, dataset_name):
    """Función para evaluar todos los modelos en un dataset específico."""
    results = {}
    print(f"\n--- INICIANDO EVALUACIÓN EN DATASET {dataset_name.upper()} ---")
    if not MODELS_DIR:
        print("No se puede evaluar porque no se encontró el directorio de modelos.")
        return results

    for name, model_file in models_to_evaluate.items():
        path = os.path.join(MODELS_DIR, model_file)
        if not os.path.exists(path):
            print(f"\nADVERTENCIA: No se encontró el modelo {name} en {path}. Saltando.")
            results[name] = None
            continue
        try:
            print(f"\nEvaluando: {name}...")
            model = YOLO(path)
            metrics = model.val(data=dataset_yaml, split='test', name=f'eval_{name}_on_{dataset_name.lower()}')
            results[name] = metrics
        except Exception as e:
            print(f"\nERROR al evaluar el modelo {name}: {e}")
            results[name] = None
    print(f"\n--- EVALUACIÓN EN {dataset_name.upper()} COMPLETADA ---")
    return results

# Ejecutar las dos evaluaciones
results_acuatico = run_evaluation(ACUATICO_YAML, 'Acuatico')


--- INICIANDO EVALUACIÓN EN DATASET ACUATICO ---

Evaluando: Aereo-Nano...
Ultralytics 8.3.228 🚀 Python-3.9.13 torch-2.8.0+cu128 CPU (AMD Ryzen 7 5800X 8-Core Processor)
YOLOv12n summary (fused): 159 layers, 2,556,923 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2440.1±879.1 MB/s, size: 30.1 KB)
val: Scanning /home/user/work/EONSEA/Articulo-Corrosion/dataset_yolo/labels/test... 21 images, 12 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 33/33 3.0Kit/s 0.0s
WARNING ⚠️ val: Cache directory /home/user/work/EONSEA/Articulo-Corrosion/dataset_yolo/labels is not writable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 1.3it/s 2.4s2.3s
                   all         33         52      0.234     0.0769     0.0443     0.0124
Speed: 1.4ms preprocess, 67.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /home/user/work/EONSEA/Articulo-Corrosion/runs/detect/e

## 3. Tablas de Resultados

A continuación se muestran los resultados consolidados en tablas.

In [4]:
def create_and_print_df(results_dict, dataset_name):
    """Crea y muestra un DataFrame simple con los resultados."""
    print(f"\n--- TABLA DE RESULTADOS: DATASET {dataset_name.upper()} ---")
    data_for_df = []
    for name, metrics in results_dict.items():
        if metrics and hasattr(metrics, 'box'):
            data_for_df.append({
                'Modelo': name,
                'mAP50-95': metrics.box.map,
                'mAP50': metrics.box.map50,
                'Precisión': metrics.box.p[0],
                'Recall': metrics.box.r[0],
                'F1-Score': metrics.box.f1[0]
            })
        else:
            # Incluir fila para modelos que no se encontraron o fallaron
            data_for_df.append({'Modelo': name, 'mAP50-95': 0, 'mAP50': 0, 'Precisión': 0, 'Recall': 0, 'F1-Score': 0})

    if not data_for_df:
        print("No se generaron resultados.")
        return

    df = pd.DataFrame(data_for_df).sort_values(by='mAP50', ascending=False).reset_index(drop=True)
    print(df.to_string(float_format='{:.4f}'.format))

# Mostrar ambas tablas
create_and_print_df(results_acuatico, 'Acuatico')


--- TABLA DE RESULTADOS: DATASET ACUATICO ---
            Modelo  mAP50-95  mAP50  Precisión  Recall  F1-Score
0    Acuatico-Nano    0.4364 0.7348     0.9047  0.5479    0.6825
1  Acuatico-Medium    0.3960 0.6724     0.7797  0.5769    0.6632
2     Mixto-Medium    0.2137 0.4414     0.4970  0.4231    0.4571
3   Acuatico-Large    0.1968 0.4320     0.5098  0.4038    0.4507
4       Mixto-Nano    0.2113 0.3850     0.5582  0.3654    0.4417
5       Aereo-Nano    0.0124 0.0443     0.2336  0.0769    0.1157
6      Aereo-Large    0.0093 0.0349     0.3462  0.0385    0.0692
7     Aereo-Medium    0.0126 0.0218     0.1671  0.0577    0.0858
8      Mixto-Large    0.0000 0.0000     0.0000  0.0000    0.0000
